### Домашнее задание Transformers Training (50 баллов)

В этом домашнем задании требуется обучить несколько Transformer-based моделей в задаче машинного перевода. Для обучения можно воспользоваться текущим проектом, так и реализовать свой пайплайн обучения. Если будете использовать проект, теги **TODO** проекта отмечают, какие компоненты надо реализовать.
В ноутбуке нужно только отобразить результаты обучения и выводы. Архитектура модели(количетсво слоев, размерность и тд) остается на ваш выбор.

Ваш код обучения нужно выложить на ваш github, в строке ниже дать ссылку на него. В первую очередь будут оцениваться результаты в ноутбуке, код нужен для проверки адекватности результатов. 

Обучать модели до конца не нужно, только для демонстрации, что модель обучается и рабочая - снижение val_loss, рост bleu_score.

### Данные

`
wget https://www.manythings.org/anki/rus-eng.zip && unzip rus-eng.zip
`

Модели нужно обучить на задаче перевода с английского на русский. 

#### Сcылка на ваш github с проектом(вставить свой) - https://github.com/runnerup96/pytorch-machine-translation

Ноутбук с результатами выкладывать на ваш **google диск** курса. 

In [1]:
# !wget https://www.manythings.org/anki/rus-eng.zip && unzip rus-eng.zip

In [2]:
from datamanip import get_dataset, get_tokenizer,compute_metrics, preprocess_function, DatasetConfig, collate_fn
from dataclasses import dataclass
from torch.utils.data import Dataset
from transformers import AutoTokenizer  
from datasets import Dataset
import numpy as np
from datasets import load_metric
import wandb


/home/user/conda/envs/runfme_default/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jovyan/runfme/triton/translate/datamanip.py:71: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/home/user/conda/envs/runfme_default/lib/python3.11/site-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `tru

In [3]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

In [4]:
from omegaconf import OmegaConf
from functools import partial

conf = OmegaConf.load('./cfg_tune_t5.yaml')
dataset_cfg = OmegaConf.structured(DatasetConfig(**conf['dataset']))

tokenizer = AutoTokenizer.from_pretrained('./my_tokenizer')
import datamanip
datamanip.tokenizer = tokenizer
pad_token_id = tokenizer.pad_token_id
eos_token_id = tokenizer.eos_token_id

dataset = get_dataset(dataset_cfg)
dataset = dataset.train_test_split(test_size=0.03)
tokenized_dataset = dataset.map(partial(preprocess_function, add_eos=True), batched=True)

Map: 100%|██████████| 9672/9672 [00:00<00:00, 29711.04 examples/s]


In [5]:
import torch
import torch.nn as nn
from dataclasses import dataclass
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers.modeling_outputs import Seq2SeqLMOutput
from model import MyEncoderDecoderModelForSeq2SeqLM, ModelConfig

model_new = MyEncoderDecoderModelForSeq2SeqLM(
    ModelConfig(vocab_size=len(tokenizer), pad_token=pad_token_id, eos_token=eos_token_id)
)
model_new.cuda()

MyEncoderDecoderModelForSeq2SeqLM(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-4): 5 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0): TransformerDecoderLayer(
          (self_attn): Multih

In [6]:
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

EPOCH_NUM = 50
train_dl = DataLoader(tokenized_dataset['train'], batch_size=200, shuffle=True, collate_fn=collate_fn)
test_dl = DataLoader(tokenized_dataset['test'], batch_size=200, shuffle=False, collate_fn=collate_fn)
optimizer = optim.Adam(model_new.parameters(), lr=3e-4)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, len(train_dl) * EPOCH_NUM)

In [8]:
import wandb
from tqdm import tqdm
import os
os.environ["WANDB_API_KEY"]= '844ae81dd9f3dac384cf5cdb478d4e939ed71fa1'

wandb.init()

step = 0
for epoch in range(EPOCH_NUM):
    print('Train')
    for i, train_batch in enumerate(tqdm(train_dl)):
        res = model_new(**{k: v.cuda() for k,v in train_batch.items() if k in ['input_ids', 'labels','attention_mask']})
        # backward pass
        optimizer.zero_grad()
        res.loss.backward()
        # update weights
        optimizer.step()
        scheduler.step()
        # print(res.loss)
        if step%50 == 0:
            wandb.log({'lr': optimizer.param_groups[0]['lr'], 'loss': res.loss.item()}, step)
        step+=1
        
    print('Eval crossentropy')
    losses = []
    with torch.no_grad():
        for i, test_batch in tqdm(enumerate(test_dl)):
            res = model_new(**{k: v.cuda() for k,v in test_batch.items() if k in ['input_ids', 'labels','attention_mask']})
            losses.append(res.loss.item())
    wandb.log({'val_loss': np.mean(losses)}, step)
    
    if (epoch+1) % 10 == 0  or epoch == EPOCH_NUM - 1:
        print('Eval bleu')
        preds = []
        targets = [] 
        with torch.no_grad():
            for test_sample in tqdm(tokenized_dataset['test']):
                targets.append(test_sample['labels'])
                result = model_new.generate(torch.tensor(test_sample['input_ids']).cuda(), max_length=30)
                preds.append(result.cpu())
                
        max_target_len = max([len(seq) for seq in targets])
        padded_targets = [seq + [-100] * (max_target_len - len(seq)) for seq in targets]
        bleu_score = compute_metrics((torch.stack(preds),torch.tensor(padded_targets)))        
        print(f'BLEU: {bleu_score}')
        wandb.log({'bleu_score': bleu_score}, step)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: runfme. Use `wandb login --relogin` to force relogin


Train


 48%|████▊     | 743/1564 [00:17<00:18, 44.89it/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7efcc4f18b90>>
Traceback (most recent call last):
  File "/home/user/conda/envs/runfme_default/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
100%|██████████| 1564/1564 [00:36<00:00, 43.34it/s]


Eval crossentropy


49it [00:00, 67.71it/s]


Train


  5%|▌         | 85/1564 [00:02<00:35, 42.20it/s]


KeyboardInterrupt: 

In [7]:
# model_new.load_state_dict(torch.load('./from_scratch_10ep.pt'))
# torch.save(model_new.state_dict(), './from_scratch_10ep.pt')

<All keys matched successfully>

In [11]:
for i in dataset['test'].select(range(10)):
    source, target = i['source'], i['target']
    inputs = tokenizer(
        source,
        return_tensors='pt',
        max_length=256,
        truncation=True
    )

    # Get correct sentence ids.
    corrected_ids = model_new.generate(
        inputs.input_ids[0].cuda(),
        max_length=256
    )

    # Decode.
    res = tokenizer.decode(
        corrected_ids,
        skip_special_tokens=True
    )
    print(f'Eng: {source}')
    print(f'Reference: {target}')
    print(f'Predicted: {res}')

Eng: He chose not to run for the presidential election.
Reference: Он решил не принимать участия в президентских выборах.
Predicted: Он решил не на выборах президента.
Eng: I don't feel sorry for her.
Reference: Мне её не жалко.
Predicted: Мне не жалко. Я что вам не жалко. сочувствую её.
Eng: I think that Tom won't go to Boston with Mary.
Reference: Я думаю, Том не поедет с Мэри в Бостон.
Predicted: Думаю, Том не поедет в Бостон с Мэри.
Eng: Please bring us two cups of coffee.
Reference: Пожалуйста, принесите нам две чашки кофе.
Predicted: Зайдите мы кофе кофе кофе.
Eng: Tom is Mary's half-brother.
Reference: Том — единокровный брат Мэри.
Predicted: Том ведь подаст Мэри - племяннице дам друг у Мэри.
Eng: The rat made a hole in the wall.
Reference: Крыса прогрызла дыру в стене.
Predicted: В стене виски голыми.
Eng: I'm a bus driver.
Reference: Я водитель автобуса.
Predicted: Я автобус водите водитель автобуса. Я водитель автобуса.
Eng: Can I have this film developed?
Reference: Не могли